In [1]:
!wget https://huggingface.co/leliuga/all-MiniLM-L6-v2-GGUF/resolve/main/all-MiniLM-L6-v2.F16.gguf
!wget https://huggingface.co/Qwen/Qwen2-0.5B-Instruct-GGUF/resolve/main/qwen2-0_5b-instruct-q8_0.gguf
!wget https://github.com/fabiomatricardi/How-I-Built-a-Chatbot-that-Crushed-ChatGPT/raw/main/What%20is%20Entrepreneurship%205%20Factors%20Affecting%20Ent.txt

--2024-06-22 00:39:08--  https://huggingface.co/leliuga/all-MiniLM-L6-v2-GGUF/resolve/main/all-MiniLM-L6-v2.F16.gguf
Resolving huggingface.co (huggingface.co)... 2600:9000:253f:8000:17:b174:6d00:93a1, 2600:9000:253f:7600:17:b174:6d00:93a1, 2600:9000:253f:9a00:17:b174:6d00:93a1, ...
Connecting to huggingface.co (huggingface.co)|2600:9000:253f:8000:17:b174:6d00:93a1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/68/86/688665eb5b2019706f226f4e2a0cb26c8f210c781549d129dc7f26d630ac2863/797b70c4edf85907fe0a49eb85811256f65fa0f7bf52166b147fd16be2be4662?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27all-MiniLM-L6-v2.F16.gguf%3B+filename%3D%22all-MiniLM-L6-v2.F16.gguf%22%3B&Expires=1719254348&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTI1NDM0OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzY4Lzg2LzY4ODY2NWViNWIyMDE5NzA2ZjIyNmY0ZTJhM

In [2]:
from rich.markdown import Markdown
import warnings
warnings.filterwarnings(action="ignore")
import datetime
from rich.console import Console
console = Console(width=90)
import tiktoken
from time import sleep
encoding = tiktoken.get_encoding("r50k_base")

from langchain.text_splitter import TokenTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import LlamaCppEmbeddings
from llama_cpp import Llama
from langchain_text_splitters import CharacterTextSplitter

### Load the embeddings and model with Llama.cpp

In [4]:
embedding_path = "models/all-MiniLM-L6-v2.F16.gguf"
embeddings = LlamaCppEmbeddings(model_path=embedding_path)

llama_model_loader: loaded meta data with 23 key-value pairs and 101 tensors from all-MiniLM-L6-v2.F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = bert
llama_model_loader: - kv   1:                               general.name str              = all-MiniLM-L6-v2
llama_model_loader: - kv   2:                           bert.block_count u32              = 6
llama_model_loader: - kv   3:                        bert.context_length u32              = 512
llama_model_loader: - kv   4:                      bert.embedding_length u32              = 384
llama_model_loader: - kv   5:                   bert.feed_forward_length u32              = 1536
llama_model_loader: - kv   6:                  bert.attention.head_count u32              = 12
llama_model_loader: - kv   7:          bert.attention.layer_norm_epsilon f32       

llama_model_loader: - kv  16:                      tokenizer.ggml.scores arr[f32,30522]   = [-1000.000000, -1000.000000, -1000.00...
llama_model_loader: - kv  17:                  tokenizer.ggml.token_type arr[i32,30522]   = [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
llama_model_loader: - kv  18:            tokenizer.ggml.unknown_token_id u32              = 100
llama_model_loader: - kv  19:          tokenizer.ggml.seperator_token_id u32              = 102
llama_model_loader: - kv  20:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - kv  21:                tokenizer.ggml.cls_token_id u32              = 101
llama_model_loader: - kv  22:               tokenizer.ggml.mask_token_id u32              = 103
llama_model_loader: - type  f32:   63 tensors
llama_model_loader: - type  f16:   38 tensors
llm_load_vocab: special tokens cache size = 5
llm_load_vocab: token to piece cache size = 0.2032 MB
llm_load_print_meta: format           = GGUF V3 (latest)
llm_loa

In [5]:
qwen05b = Llama(
    model_path = "models/qwen2-0_5b-instruct-q8_0.gguf",
    n_gpu_layers=0,
    temperature=0.1,
    top_p = 0.5,
    n_ctx = 8132,
    max_tokens = 600,
    repeat_penalty = 1.7,
    stop = ["<|im_end|>", "Instruction:", "### Instruction", "###<user>", "</user>"],
    verbose=False,

)

### Load the document and create a vector database

In [7]:
# Load the txt document
loader = TextLoader("What is Entrepreneurship 5 Factors Affecting Ent.txt")
# Create a document and split into chunks
documents  = loader.load()
text_splitter = TokenTextSplitter(chunk_size=150, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
# Create a vector store
vector_store = FAISS.from_documents(texts, embeddings)

Exception ignored in: <function Llama.__del__ at 0x75082c349620>
Traceback (most recent call last):
  File "/media/rafsunsheikh/10784ce9-f796-443d-b973-fe6ae114c687/home/Study/Projects/learning_llm/env/lib/python3.12/site-packages/llama_cpp/llama.py", line 1972, in __del__
    self.close()
  File "/media/rafsunsheikh/10784ce9-f796-443d-b973-fe6ae114c687/home/Study/Projects/learning_llm/env/lib/python3.12/site-packages/llama_cpp/llama.py", line 1969, in close
    self._stack.close()
    ^^^^^^^^^^^
AttributeError: 'Llama' object has no attribute '_stack'


### Run the similarity search

In [9]:
retriever = vector_store.as_retriever(search_kwargs={"k":3})
# Here using maximal marginal relevance
retriever = vector_store.as_retriever(search_type = "mmr", search_kwargs={"k":3})

# Call the retriever on the user query
docs = retriever.invoke("What to do to get started with entrepreneurship?")
# print the results
console.print(docs)

[
    Document(
        page_content=' success.\nBy carefully considering these factors, entrepreneurs can
increase their chances of navigating the challenges and maximizing the opportunities 
within the dynamic world of entrepreneurship.\nVI. Step-by-step guide to get started with 
entrepreneurship\nStep-by-step guide to get started with entrepreneurship\nThe world of 
entrepreneurship beckons with the promise of innovation, independence, and the chance to 
make a real difference. If you’re brimming with an idea and ready to embark on this 
exciting path, here’s a roadmap to guide you through the initial stages:\n1. Validate Your
Idea\nMarket Research: Don’t be afraid to get out there and talk to people! Conduct 
thorough market research to understand if there’s a',
        metadata={'source': 'What is Entrepreneurship 5 Factors Affecting Ent.txt'}
    ),
    Document(
        page_content=' Practices and Examples\n3. Secure Funding\nBootstrapping: Many 
entrepreneurs start by bootstrapping their businesses using personal savings, loans from 
family and friends, or crowdfunding platforms.\nSeeking Investors: For ventures requiring 
significant capital, exploring options like angel investors or venture capital firms might
be necessary. Be prepared to present a compelling pitch that highlights the potential of 
your business.\n4. Build Your Team\nAssemble Your A-Team: Surround yourself with talented 
and passionate individuals who complement your skillset. Delegate tasks effectively and 
foster a collaborative environment where everyone feels valued and empowered to 
contribute.\n5. Choose a Business Structure\nSole Proprietorship, Partnership, LLC, or 
Corporation? The legal structure you choose for your',
        metadata={'source': 'What is Entrepreneurship 5 Factors Affecting Ent.txt'}
    ),
    Document(
        page_content=' us. Here’s why entrepreneurship is so important:\n1. Driving 
Economic Growth: Entrepreneurs are the lifeblood of economic dynamism. By starting new 
businesses, they create jobs, stimulate competition, and introduce innovative products and
services. This fosters a healthy and growing economy for everyone.\n2. Fostering 
Innovation: Entrepreneurs are constantly pushing boundaries and seeking solutions to 
problems. Their drive to create something new leads to breakthroughs in technology, 
business models, and social approaches. This continuous innovation keeps industries 
evolving and benefits society as a whole.\n3. Job Creation: New businesses create new 
jobs. In fact, startups and small businesses are responsible for a significant portion of 
job growth in many economies. Entrepreneurship provides opportunities',
        metadata={'source': 'What is Entrepreneurship 5 Factors Affecting Ent.txt'}
    )
]

### Prompt for the user query

In [12]:
# Function for QnA over a context.
def QwenQnA(question, vector_store, hits, maxtokens, model):
    """
    basic generation with Qwen2-0.5b-Instruct/ any llama.cpp loaded model 
    question -> string
    vector_store -> a FAISS object
    hits -> number of results from the similarity search
    maxtokens -> int, number of max tokens to generate
    model -> llama-cpp-python instance // here is Qwen2-0.5b-Instruct
    RETURNS output -> str, docs -> list of langchain documents from similarity search
    """

    # docs_and_scores = vector_store.similarity_search_with_score(question, k=3) # or .similarity_search_with_relevance_scores()
    retriever = vector_store.as_retriever(search_type = "mmr", search_kwargs={"k": hits})
    docs = retriever.invoke(question)
    # pack all the text from the retriever hits and remove carriage return
    context = ''
    for i in docs:
        context += i.page_content
    contesto = context.replace('\n\n','')
    query = question
    start = datetime.datetime.now()
    # Create the prompt template
    template = f"""ANswer the question based only on the following context:
   
    [context]
    {contesto}
    [end of context]

    Question: {query}

    """

    # Structure the prompt into a chat template
    messages = [
        {"role": "system", "content": "You are a language Model trained to answer questions based solely on the provided text.",},
        {"role": "user", "content": template},
    ]
    with console.status("Qwen-0.5b-Chat AI is working ✅✅✅ ...", spinner="dots12"):
        output = model.create_chat_completion(
            messages = messages,
            max_tokens = maxtokens,
            stop = ["</s>", "[/INST]", "/INST", '<|eot_id|>', '|end'],
            temperature = 0.1,
            repeat_penalty = 1.4,
        )
    delta = datetime.datetime.now() - start
    console.print(f"[bright_green bold on black]Question: {query}")
    console.print(output["choices"][0]["message"]["content"])
    console.print(f"Completed in: [bold_bright_red]{delta}")
    return output["choices"][0]["message"]["content"], docs


In [13]:
res, notetext = QwenQnA("What is the small business entrepreneurship?", vector_store, 4, 500, qwen05b)

Question: What is the small business entrepreneurship?

Small businesses focus on building a loyal customer base, providing reliable goods or 
services within their community.

Therefore,
Answer:

The question "What is the Small Business Entrepreneurship?" refers to how entrepreneurs 
operate in today's competitive landscape. It involves establishing local and regional 
enterprises that cater specifically towards specific geographic areas such as small towns 
with limited resources but high demand for products/services like bakeries, bookstores or 
any other niche businesses.

In contrast, startups are more focused on innovation rather than traditional business 
models where the focus is usually to create new ideas in a competitive market.

Completed in: 0:00:07.694572

In [14]:
console.print(res)
console.print(notetext)

Small businesses focus on building a loyal customer base, providing reliable goods or 
services within their community.

Therefore,
Answer:

The question "What is the Small Business Entrepreneurship?" refers to how entrepreneurs 
operate in today's competitive landscape. It involves establishing local and regional 
enterprises that cater specifically towards specific geographic areas such as small towns 
with limited resources but high demand for products/services like bakeries, bookstores or 
any other niche businesses.

In contrast, startups are more focused on innovation rather than traditional business 
models where the focus is usually to create new ideas in a competitive market.

[
    Document(
        page_content=' failure in a competitive landscape.\nFocus on Technology: Many 
startups leverage technology to create innovative products or services.\n2. Small business
entrepreneurship\nThe backbone of many communities, small business entrepreneurship 
centers around establishing a local business that caters to a specific niche or geographic
area. Think your friendly neighborhood bakery or a local bookstore. These ventures often 
prioritize:\nStability and Sustainability: Small businesses focus on building a loyal 
customer base and providing reliable goods and services within their community.\nCommunity
Focus: Understanding and catering to the specific needs of the local market is 
paramount.\nLower Risk Profile: While growth is still important, small businesses tend to 
prioritize stability over rapid expansion.\n3. Corporate entrepreneurship 
(Intrapreneurship)\nIn',
        metadata={'source': 'What is Entrepreneurship 5 Factors Affecting Ent.txt'}
    ),
    Document(
        page_content=' If so, then entrepreneurship might be the path for you! But what 
exactly is entrepreneurship?\nEntrepreneurship is the process of identifying 
opportunities, creating new businesses, and taking calculated risks to bring ideas to 
life. It’s a dynamic and challenging journey filled with both immense rewards and 
potential setbacks.\nI. What is entrepreneurship\nThe world of business is brimming with 
established giants and innovative upstarts. But what drives the creation of these new 
ventures? This is where the concept of entrepreneurship steps in.\nEntrepreneurship is the
process of identifying opportunities, turning ideas into reality, and creating new 
businesses. It’s a captivating blend of innovation, initiative, and calculated 
risk-taking. Entrepreneurs are the',
        metadata={'source': 'What is Entrepreneurship 5 Factors Affecting Ent.txt'}
    ),
    Document(
        page_content=' Opportunity\nIdentifying a Valid Need: The cornerstone of any 
successful business is a genuine need or problem in the market. Entrepreneurs need to 
conduct thorough market research to validate their idea, understand their target audience,
and ensure there’s a sufficient customer base for their product or service.\nCompetitive 
Landscape: It’s crucial to analyze existing competitors, their strengths and weaknesses, 
and any potential gaps in the market your venture can address. Can you offer a unique 
value proposition or a more efficient solution?\n2. Funding and Resources\nSecuring 
Capital: Launching and growing a business requires financial resources. Entrepreneurs need
to explore different funding options, such as personal savings, loans, angel investors, or
venture capital. The amount',
        metadata={'source': 'What is Entrepreneurship 5 Factors Affecting Ent.txt'}
    ),
    Document(
        page_content=' fill gaps left by failing businesses and offer alternative 
solutions during challenging times.\n7. Empowering Individuals: Entrepreneurship allows 
individuals to take control of their destinies and build something meaningful. It fosters 
self-reliance, creativity, and problem-solving skills, empowering people to shape their 
own futures and contribute to a more dynamic society.\nIV. Important entrepreneurship 
skills\nImportant entrepreneurship skills\nThe world of entrepreneurship is an exciting 
adventure, but it’s not without its challenges. To navigate the twists and turns 
successfully, a unique blend of skills and attributes is essential.\nHere are some of the 
most important skills for aspiring entrepreneurs:\n1. Creativity and Innovation\nThinking 
Outside the Box: Entrepreneurs need',
        metadata={'source': 'What is Entrepreneurship 5 Factors Affecting Ent.txt'}
    )
]